In [72]:
!pip install JSAnimation matplotlib jsonschema dataclasses

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [22]:
import json

with open("board_00.json") as f:
    json_board_infos = json.load(f)
json_board_infos

{'map': {'x_max': 10, 'y_max': 6, 'max_range': 4, 'max_rounds': 50},
 'bot': {'x_pos': 1, 'y_pos': 1},
 'trash': [{'x_pos': 4, 'y_pos': 5},
  {'x_pos': 10, 'y_pos': 1},
  {'x_pos': 10, 'y_pos': 3}]}

In [24]:
from jsonschema import validate


# https://github.com/Julian/jsonschema
schema = {
    "type" : "object",
    "properties" : {
        "map" : {
            "type" : "object",
            "properties": {
                "x_max": {"type": "number"},
                "y_max": {"type": "number"},
                "max_range": {"type": "number"},
                "max_rounds": {"type": "number"},
            }
        },
        "bot" : {
            "type" : "object",
            "properties": {
                "x_pos": {"type": "number"},
                "y_pos": {"type": "number"},
            }
        },
        "trash": {
            # https://json-schema.org/understanding-json-schema/reference/array.html
            "type": "array",
            "items": {
                "type": "object",
                 "properties": {
                    "x_pos": {"type": "number"},
                    "y_pos": {"type": "number"},
                }   
            }
        }
    },
}

validate(json_board_infos, schema)

In [289]:
"""
https://github.com/ericvsmith/dataclasses
https://www.python.org/dev/peps/pep-0557/
https://realpython.com/python-data-classes/
"""
from dataclasses import dataclass, field
from enum import Enum
import random
from typing import Dict, List


class ITEMS_IN_GRID(Enum):
    empty = 0
    trash = 1
    bot = 2
    

@dataclass
class Position(object):
    x_pos: int
    y_pos: int
        
    @staticmethod
    def load(json: dict):
        return Position(json['x_pos'], json['y_pos'])
    
    @property
    def i(self):
        return self.x_pos - 1
    
    @property
    def j(self):
        return self.y_pos - 1
    
    @property
    def ndarray_indices(self):
        return self.i, self.j
    
    def __add__(self, other):
        return Position(
            x_pos=self.x_pos + other.x_pos,
            y_pos=self.y_pos + other.y_pos,
        )
    
    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return self.__add__(other)

@dataclass
class Dimension:
    x_max: int
    y_max: int
        
    @staticmethod
    def load(json: dict):
        return Dimension(json['x_max'], json['y_max'])
    
    def __iter__(self):
        yield self.x_max
        yield self.y_max

@dataclass
class Trash:
    position : Position

    @staticmethod
    def load(json: dict):
        return Trash(Position(json['x_pos'], json['y_pos']))

@dataclass
class Trashs:
    trashs: Dict[Position, Trash]
    #
    rate_expand : float = 1.0/4.0
        
    @staticmethod
    def load(json):
        return Trashs(trashs=[
            trash
            for trash in map(Trash.load, json)
        ])
    
    def __iter__(self):
        return iter(self.trashs)
    
    def __len__(self):
        return len(self.trashs)
    
    def propage(self):
        return np.array(self.trashs)[np.random.rand(len(self.trashs)) >= self.rate_expand]
    
    def add(self, trash: Trash):
        self.trashs.append(trash)

@dataclass
class Bot:
    position: Position
    
    @staticmethod
    def load(json):
        return Bot(position=Position.load(json))
    
    @property
    def ndarray_indices(self):
        return self.position.ndarray_indices
        

neighboors_dir = [
    Position(-1, 0),
    Position(+1, 0),
    Position(0, -1),
    Position(0, +1),
]


@dataclass
class Grid:
    dimension: Dimension
    grid: np.ndarray
        
    @staticmethod
    def load(json: dict):
        dimension = Dimension.load(json)
        return Grid(
            dimension=dimension,
            grid = np.zeros(tuple(dimension), dtype=int)
        )
    
    def add_trash(self, trash: Trash):
        self.grid[trash.position.i, trash.position.j] = ITEMS_IN_GRID.trash.value
        
    def add_trash_from_position(self, position: Position):
        self.grid[position.i, position.j] = ITEMS_IN_GRID.trash.value
        
    def set_bot(self, bot: Bot):
        self.grid[bot.ndarray_indices] = ITEMS_IN_GRID.bot.value
        
    def is_in_grid(self, position: Position) -> bool:
        return (1 <= position.x_pos <= self.dimension.x_max) and (1 <= position.y_pos <= self.dimension.y_max)
    
    def is_not_trash(self, position: Position) -> bool:
        return self.grid[position.ndarray_indices] != ITEMS_IN_GRID.trash.value
    
    def free_neighboors(self, position: Position) -> List[Position]:
        return filter(
            self.is_not_trash,
            filter(
                self.is_in_grid,
                [
                    position + neighboor_dir
                    for neighboor_dir in neighboors_dir
                ]
            )
        )
    
    
@dataclass
class Board:
    max_range: int
    max_rounds: int
    #
    grid: Grid
    trashs: Trashs
    bot: Bot

    def __post_init__(self):
        for trash in self.trashs:
            self.grid.add_trash(trash)
        self.grid.set_bot(self.bot)
            
    @staticmethod
    def load(json: dict):
        return Board(
            max_range = json['map']['max_range'],
            max_rounds = json['map']['max_rounds'],
            grid = Grid.load(json['map']),
            bot = Bot.load(json['bot']),
            trashs = Trashs.load(json['trash'])
        )
    
    def propage_trashs(self):
        # on parcourt la liste des trashs qui se propagent
        for trash_propaged in self.trashs.propage():
            # on recupère pour chaque trash à propager la liste des directions possibles de propagation
            # on en choisit une (au hasard)
            try:
                trash_position_to_propagate = random.choice(list(self.grid.free_neighboors(trash_propaged.position)))
            except IndexError:
                pass
            else:
                trash_to_propagate = Trash(position=trash_position_to_propagate)
                self.grid.add_trash(trash_to_propagate)
                self.trashs.add(trash_to_propagate)

In [290]:
board = Board.load(json_board_infos)
board

Board(max_range=4, max_rounds=50, grid=Grid(dimension=Dimension(x_max=10, y_max=6), grid=array([[2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0]])), trashs=Trashs(trashs=[Trash(position=Position(x_pos=4, y_pos=5)), Trash(position=Position(x_pos=10, y_pos=1)), Trash(position=Position(x_pos=10, y_pos=3))], rate_expand=0.25), bot=Bot(position=Position(x_pos=1, y_pos=1)))

In [279]:
for i in range(board.max_range):
    print(f"len(trashs): {len(board.trashs)}")
    board.propage_trashs()
    print(board.grid)

len(trashs): 3
Grid(dimension=Dimension(x_max=10, y_max=6), grid=array([[2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0]]))
len(trashs): 5
Grid(dimension=Dimension(x_max=10, y_max=6), grid=array([[2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0]]))
len(trashs): 8
Grid(dimension=Dimension(x_max=10, y_max=6), grid=array([[2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1